# VANI

**Initial notebook configuration**

In [1]:
%load_ext autoreload
# %load_ext snakeviz
# %load_ext jupyterflame

In [2]:
import os
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
sys.path.append(os.path.dirname(os.getcwd()))

/usr/workspace/iopp/projects/vani-analysis-tool/vani-env/bin/python3
3.7.2 (default, Feb 26 2019, 08:59:10) 
[GCC 4.9.3]
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)


In [3]:
# Filter unnecessary warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

**Import libraries**

In [4]:
from IPython.display import HTML
from vani.analyzer import Analyzer, ClusterOptions, ClusterType

**Initialize VANI Analyzer**

In [5]:
%autoreload 2

# Initialize analyzer
co = ClusterOptions(cluster_type=ClusterType.Local)
vn = Analyzer(n_workers=8, cluster_options=co, debug=True)

/usr/workspace/iopp/projects/vani-analysis-tool/vani-env/lib/python3.7/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35622 instead
  f"Port {expected} is already in use.\n"


Dashboard link: http://127.0.0.1:35622/status


**Analyze logs**

In [6]:
%%time
%autoreload 2

# Analysis configuration
# log_dir = "/p/gpfs1/iopp/parquet_app_logs/hacc/nodes-32/workflow-0"
# log_dir = "/p/gpfs1/iopp/parquet_app_logs/cm1/nodes-32/workflow-4"
# log_dir = "/p/gpfs1/iopp/parquet_app_logs/lbann-cosmoflow/nodes-32"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/montage_pegasus/nodes-32/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"

# log_dir = "/p/lustre1/iopp/parquet_app_logs/hacc/nodes-32/workflow-0"
log_dir = "/p/lustre1/iopp/parquet_app_logs/cm1/nodes-32/workflow-4"
# log_dir = "/p/lustre1/iopp/parquet_app_logs/lbann-cosmoflow/nodes-32"
# log_dir = "/p/lustre1/iopp/recorder_app_logs/montage_pegasus/nodes-32/_parquet"
# log_dir = "/p/lustre1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"

# Do the analysis
analysis = vn.analyze_parquet_logs(log_dir, max_depth=10)

Logs read (0.5621045744046569)
---------------
Job time: 668.306
I/O time: 667.805
Max duration: 0.007
Mean BW: 565.574
Total ranks: 1280.000
Total size: 21.178
Total files: 774.000
---------------


distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)


CPU times: user 1min 26s, sys: 6.68 s, total: 1min 32s
Wall time: 2min 48s


In [7]:
%%time
analysis.render_tree()

Analysis
└── Timeline Analysis
    ├── [0.000-668.306] Size=2 21.17757 100.0% | Bandwidth=1 565.57407 100.0% | Parallelism=2 1280.00000 100.0% | Files=2 774.00000 100.0%
    │   ├── [0.000-334.153] Size=2 20.46060 96.6% | Bandwidth=1 849.86937 150.3% | Parallelism=2 1280.00000 100.0% | Files=1 376.00000 48.6%
    │   │   ├── [0.000-167.076] Size=2 20.31332 95.9% | Bandwidth=1 1119.72849 198.0% | Parallelism=2 1280.00000 100.0% | Files=1 186.00000 24.0%
    │   │   │   ├── [0.000-83.538] Size=2 20.17161 95.2% | Bandwidth=1 1341.21215 237.1% | Parallelism=2 1280.00000 100.0% | Files=1 88.00000 11.4%
    │   │   │   │   ├── [0.000-41.769] Size=2 20.16870 95.2% | Bandwidth=1 1482.46962 262.1% | Parallelism=2 1280.00000 100.0% | Files=1 40.00000 5.2%
    │   │   │   │   │   ├── [0.000-20.885] Size=2 20.16773 95.2% | Bandwidth=1 1536.40965 271.7% | Parallelism=2 1280.00000 100.0% | Files=1 24.00000 3.1%
    │   │   │   │   │   │   ├── [0.000-10.442] Size=2 20.16761 95.2% | Bandwidth=1 1543.4

In [8]:
%%time
%autoreload 2
filter_group_node = analysis.children[0]
vn.save_filter_group_node_as_flamegraph(filter_group_node=filter_group_node, output_path="flamegraph.txt")

CPU times: user 17.7 ms, sys: 1.85 ms, total: 19.6 ms
Wall time: 23.3 ms


In [9]:
%%time
! chmod +x flamegraph.pl
! flamegraph.pl flamegraph.txt > flamegraph.svg

CPU times: user 58 ms, sys: 45.2 ms, total: 103 ms
Wall time: 623 ms


In [22]:
HTML(filename="flamegraph.svg")